In [7]:
%%writefile util.py

import os
import glob
import pandas as pd
import numpy as np
import shutil
from math import sqrt
from numpy import concatenate
from collections import namedtuple
from matplotlib import pyplot
import matplotlib.pyplot as plt
import tensorflow as tf
import keras
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM,GRU
from keras.models import model_from_json
from scipy.stats import gaussian_kde
from scipy.spatial import distance
from sklearn.cluster import KMeans
from sklearn import preprocessing
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_squared_error,mean_absolute_error
from sklearn.cluster import SpectralClustering
import tqdm
from tqdm import tqdm
from sklearn.metrics import silhouette_samples, silhouette_score
import matplotlib.cm as cm
from joblib import dump, load




def save_model(model, name):
    print('SAVING model: {0}'.format(name))    
    json_string = model.to_json()
    architecture = name+'_architecture.json' 
    weights = name+'_weights.h5'
    open(architecture, 'w').write(json_string)
    model.save_weights(weights)


def retrieve_model(name, weights=True):
    print('RETRIEVING model: {0}'.format(name))
    architecture = name + '_architecture.json' 
    model_saved = model_from_json(open(architecture).read())
    
    if weights:
        weights = name+'_weights.h5'    
        model_saved.load_weights(weights)
    return model_saved


    
def train_by_cluster(cluster,cluster_users,hps):
    
    standardize = False
    count = len(cluster_users)    
    print('Training model: {0}:: users: {1}'.format(cluster,len(cluster_users)))
    if(count > 30):
        
        if(hps.mix_std):
            if(count > 200):
                standardize = False
            else:
                standardize = True
                
        else:
            standardize = hps.standardize
        
        train_files = get_files(cluster_users,hps.train_file)
        train_X, train_y  = get_cluster_data(train_files,hps,standardize,True,cluster)
        print('TRAIN:: ',train_X.shape,train_y.shape)

        epochs = hps.epochs 
        if(hps.epochs > 10):
            count = len(cluster_users)
            if(count > 400):
                epochs = 100
            elif(count  > 200):
                epochs = 350
            elif(count  > 75):
                epochs = 400

        # design network
        model = get_lstm(hps,train_X,hps.dropout[0])

        validate_files = get_files(cluster_users,hps.validate_file)
        if(validate_files):
            validate_X, validate_y  = get_cluster_data(validate_files,hps,standardize,False,cluster)
            print('VALIDATE:: ',validate_X.shape,validate_y.shape)    

            # fit network
            history = model.fit(train_X, train_y, 
                                epochs=epochs, 
                                batch_size=hps.batch_size, 
                                validation_data=(validate_X, validate_y), 
                                verbose=hps.verbose_level, 
                                shuffle=False)

            if(hps.plot_eval):
                pyplot.plot(history.history['val_loss'], label='test')

        else:        
            print('VALIDATE:: NONE')    
            validate_X, validate_y = [],[]

            # fit network
            history = model.fit(train_X, train_y, 
                                epochs=epochs, 
                                batch_size=hps.batch_size, 
                                verbose=hps.verbose_level, 
                                shuffle=False)

        # plot history
        if(hps.plot_eval):
            pyplot.plot(history.history['loss'], label='train')
            pyplot.legend()
            pyplot.show()   

        model_name = 'model_{0}'.format(cluster)
        model_file = hps.model_dir+model_name
        save_model(model,model_file)

    
def test_by_cluster(cluster,cluster_users,hps):
    
    standardize = False
    count = len(cluster_users)    
    print('Testing cluster: {0}:: users: {1}'.format(cluster,len(cluster_users)))
    if(count > 30):
        model_name = 'model_{0}'.format(cluster)
        model_file = hps.model_dir+model_name
        model = retrieve_model(model_file)

        
        test_files = get_files(cluster_users,hps.test_file)
        if(test_files):
            
            if(hps.mix_std):
                if(count > 200):
                    standardize = False
                else:
                    standardize = True
            else:
                standardize = hps.standardize
                
            
            test_X, test_y  = get_cluster_data(test_files,hps,standardize,False,cluster)
            print('TEST:: ',test_X.shape,test_y.shape)    

            # make a prediction
            yhat = model.predict(test_X)
            test_X = test_X.reshape((test_X.shape[0], test_X.shape[2]))

            # invert scaling for forecast
            inv_yhat = concatenate((yhat, test_X[:, 1:]), axis=1)
            inv_yhat = inv_yhat[:,0]

            # invert scaling for actual
            test_y = test_y.reshape((len(test_y), 1))
            inv_y = concatenate((test_y, test_X[:, 1:]), axis=1)
            inv_y = inv_y[:,0]    
            _a,_b,_norm = get_performace(inv_y,inv_yhat,test_X.shape[0],hps,standardize)
            return inv_y,inv_yhat,test_X.shape[0],_norm
        else:
            return None,None,None,None
    else:
        return None,None,None,None
    

def train(hps):
    try:
        shutil.rmtree(hps.model_dir) 
        os.mkdir(hps.model_dir)
    except OSError as e:
        print(e)
        pass
    
    users = get_clusters(hps)
    i = 0
    for val in users:
        train_by_cluster(i,val,hps)
        i += 1
    return users    


def test(users,hps):
    i = 0
    examples = 0
    actual = []
    pred = []
    
    user_counts    = []
    normalized_mae = []
    for val in users:
        y, yhat,samples,_norm = test_by_cluster(i,val,hps)
        if(y is not None):
            user_counts.append(len(yhat))
            normalized_mae.append(_norm)
            examples = examples + samples
            actual = np.concatenate((actual,y),axis=None)
            pred = np.concatenate((pred,yhat),axis=None)
        i += 1
        
    if(not hps.mix_std):
        get_performace(actual,pred,examples,hps,hps.standardize)
    get_weighted_average(user_counts,normalized_mae)
        
        
def get_user_file(user_id,file_name):
    u_id = str(int(user_id)).zfill(6)
    file_id = 'user_{0}'.format(u_id)
    file_path = file_name.format(file_id)
    exists = os.path.isfile(file_path)
    if exists:
        return file_path
    else:
        return '' 


def get_files(cluster_users,file_path):
    files = []
    for uid in cluster_users:
        fname = get_user_file(uid,file_path)
        if(fname is not ''):
            files.append(fname)
    return files

    
def get_clusters(hps):
    columns = ['user','gender','age','country','registered',
                'artist','track','total_sessions','avg_session_length',
                'max_session_length','median_session_length','total_session_rows'
              ]    
    complete_files = glob.glob(hps.data_file)
    df = pd.concat((pd.read_csv(f,names=columns,sep='\t') for f in complete_files))
    df = df[((df['total_session_rows'] > 500))]
    df = df[df['total_sessions'] > 30]
    df = df[df['avg_session_length'] < 20000]
    
    values = df.values
    Xpair = values[:,hps.cluster_columns]
    
    if(hps.use_spectral_clustering):
        model = SpectralClustering(n_clusters=hps.clusters, affinity='nearest_neighbors',assign_labels='kmeans')
        user_clusters = model.fit_predict(Xpair)
    else:    
        model = KMeans (n_clusters=hps.clusters, init='k-means++')
        clstrs = model.fit (Xpair)
        user_clusters = model.predict(Xpair) 
        
    users = []
    for i in range(hps.clusters):
        x = []
        a = np.where(user_clusters==i)
        arr = Xpair[a]
        for val in arr:
            x.append(val[0])
        users.append(x)    
    return users        


def get_summary_data(file_name,hps,train=True):
    columns = ['start','user','session_id','gender','age','country','registered',
               'prev_session_length','avg_session_length','session_length']
    
    complete_files = glob.glob(file_name)
    dataset = pd.concat((pd.read_csv(f,names=columns,sep='\t') for f in complete_files))
    dataset = dataset[dataset['avg_session_length'] < 20000]
    
    if(hps.standardize):
        non_std = pd.DataFrame(dataset,columns=['start','user','session_id','gender','age','country','registered'])
        std = pd.DataFrame(dataset,columns=['prev_session_length','avg_session_length','session_length'])
        names = std.columns

        scaler_name = hps.model_dir+'scale_baseline.joblib'
        if(train):
            scaler = preprocessing.StandardScaler().fit(std)
            dump(scaler,scaler_name)
        else:
            scaler = load(scaler_name)
        
        scaled_df = scaler.transform(std)
        scaled_df = pd.DataFrame(scaled_df, columns=names)   
        new_df = non_std.join(scaled_df) 
    else:
        new_df = dataset
    
    return new_df

    

def get_baseline_mae(hps):
    
    train  = get_summary_data('/home/ubuntu/data/summary/train/*.csv',hps)
    train['session_length'] = train['session_length'].astype('float64') 
    _train = train.groupby(['user'])['session_length'].mean().to_dict()
    
    test  = get_summary_data('/home/ubuntu/data/summary/test/*.csv',hps,False)

    inv_yhat = []
    inv_y = []
    # make a prediction
    for row in tqdm(test.iterrows(),total=test.shape[0]):
        try:
            user = row[1]['user']
            pred = _train[user]
            inv_yhat.append(pred)
            inv_y.append(float(row[1]['session_length']))
        except Exception as e:    
            pass

    rmse = sqrt(mean_squared_error(inv_y, inv_yhat))
    print('Test RMSE: %.3f' % rmse)
    
    mae = mean_absolute_error(inv_y, inv_yhat)
    print('Test MAE: %.3f' % mae)
    


    
def get_cluster_data(file_names,hps,standardize,train=True,cluster_id=0):
    
    columns = ['start','user','session_id','gender','age','country','registered',
               'prev_session_length','avg_session_length','session_length']
    
    dataset = pd.concat((pd.read_csv(f,names=columns,sep='\t') for f in file_names))
    dataset = dataset.dropna()
    dataset = dataset.sort_values(by=['start'])    
    
        
    if(standardize):
        non_std = pd.DataFrame(dataset,columns=['start','user','session_id','gender','age','country','registered'])
        std = pd.DataFrame(dataset,columns=['prev_session_length','avg_session_length','session_length'])
        
        names = std.columns
        scaler_name = hps.model_dir+'scale_cluster_{0}.joblib'.format(cluster_id)
        if(train):
            scaler = preprocessing.StandardScaler().fit(std)
            dump(scaler,scaler_name)
        else:
            scaler = load(scaler_name)
            
        scaled_df = scaler.transform(std)
        scaled_df = pd.DataFrame(scaled_df, columns=names)   
        new_df = non_std.join(scaled_df) 
    else:
        new_df = dataset

    values = new_df.values
    X = values[:,:-1]
    y = values[:,-1]    
    
    #3D - samples,timesteps,features
    X = X.reshape((X.shape[0], 1, X.shape[1]))
    return X,y



def get_cluster_stats_data(file_names,hps,standardize):
    
    columns = ['start','user','session_id','gender','age','country','registered',
               'prev_session_length','avg_session_length','session_length']
    
    dataset = pd.concat((pd.read_csv(f,names=columns,sep='\t') for f in file_names))
    dataset = dataset.dropna()
    dataset = dataset.sort_values(by=['start'])    
    
        
    if(standardize):
        non_std = pd.DataFrame(dataset,columns=['start','user','session_id','gender','age','country','registered'])
        std = pd.DataFrame(dataset,columns=['prev_session_length','avg_session_length','session_length'])
        
        names = std.columns
        scaler = preprocessing.StandardScaler()
        scaled_df = scaler.fit_transform(std)
        scaled_df = pd.DataFrame(scaled_df, columns=names)   
        new_df = non_std.join(scaled_df) 
    else:
        new_df = dataset

    values = new_df.values
    X = values[:,:-1]
    y = values[:,-1]    
    
    return X,y



def get_cluster_statistics(hps,standardize=True):
    print('Statistics for output variable Session Length')
    users = get_clusters(hps)
    for cluster in range(hps.clusters):    
        train_files = get_files(users[cluster],hps.train_file)
        X,y  = get_cluster_stats_data(train_files,hps,standardize)
        stats_var = y
        print('Cluster: {0}, Users: {1}, records: {2}'.format(cluster,len(users[cluster]),len(stats_var)))
        print('    Mean:{0:1f}; Variance:{1:3f}, Min: {2:3f}, Max: {3:3f}, median: {4:3f}'.format(np.mean(stats_var),np.var(stats_var),np.min(stats_var),np.max(stats_var),np.median(stats_var)))
        print('    Mean:{0:1f}; Variance:{1:3f}, Min: {2:3f}, Max: {3:3f}, median: {4:3f}'.format(np.mean(stats_var),np.var(stats_var),np.min(stats_var),np.max(stats_var),np.median(stats_var)))        



def get_layered_lstm(hps,train_X,dropout):
    model = Sequential()
    
    model.add(LSTM(hps.layer_dims, 
                   input_shape=(train_X.shape[1], 
                                train_X.shape[2]),
                   return_sequences=True,
                   dropout=dropout))
    for i in range(hps.no_layers):
        model.add(LSTM(hps.hidden_dim, 
                       dropout=dropout))
    
    model.add(Dense(1))
    model.compile(loss=hps.loss_func, 
                  optimizer=hps.optimizer)
    return model


def get_lstm(hps,train_X,dropout):
    model = Sequential()
    
    model.add(LSTM(hps.hidden_dim, 
                   input_shape=(train_X.shape[1], 
                                train_X.shape[2]),
                   dropout=dropout))
    
    model.add(Dense(1))
    model.compile(loss=hps.loss_func, 
                  optimizer=hps.optimizer)
    return model


def get_gru(hps,train_X):
    model = Sequential()
    model.add(GRU(hps.hidden_dim, 
                   input_shape=(train_X.shape[1], 
                                train_X.shape[2])))
    
    model.add(Dense(1))
    model.compile(loss=hps.loss_func, 
                  optimizer=hps.optimizer)
    return model


def get_weighted_average(users,normalized_mae):
    result = sum(x * y for x, y in zip(users, normalized_mae)) / sum(users)
    print('** METRICS :: Normalized MAE (Weighted): {0}'.format(result))    
    print(users)
    print(normalized_mae)


def get_performace(y,y_hat,samples,hps,standardize):
    # calculate RMSE
    rmse = sqrt(mean_squared_error(y, y_hat))
    mae = mean_absolute_error(y, y_hat)
    
    if(standardize):
        norm = mae/hps.baseline_mae_std
    else:    
        norm = mae/hps.baseline_mae
    
    print('METRICS :: RMSE: {0} ; MAE: {1} ; Normalized MAE: {2}'.format(rmse,mae,norm))    
#     pyplot.figure()
#     pyplot.plot(y, label='actual')
#     pyplot.plot(y_hat, label='pred')
#     pyplot.legend()
#     pyplot.show()   
    
    return rmse,mae,norm


def get_profiles(hps):
    columns = ['user','gender','age','country','registered',
                'artist','track','total_sessions','avg_session_length',
                'max_session_length','median_session_length','total_session_rows'
              ]    
    complete_files = glob.glob(hps.data_file)
    df = pd.concat((pd.read_csv(f,names=columns,sep='\t') for f in complete_files))
    df = df[((df['total_session_rows'] > 500))]
    df = df[df['total_sessions'] > 30]
    df = df[df['avg_session_length'] < 20000]
    return df


def plot_clusters(hps):
    user_clusters = []
    dataset = get_profiles(hps)
    values = dataset.values
    Xpair = values[:,hps.cluster_columns]
    
    if(hps.use_spectral_clustering):
        model = SpectralClustering(n_clusters=hps.clusters, affinity='nearest_neighbors',assign_labels='kmeans')
    else:    
        model = KMeans (n_clusters=hps.clusters, init='k-means++')
    
    user_clusters = model.fit_predict(Xpair)
    plt.scatter(Xpair[:, 0], Xpair[:, 1], c=user_clusters, s=50, cmap='viridis')

    
def plot_cluster_elbow(hps):
    dataset = get_profiles(hps)
    values = dataset.values
    Xpair = values[:,hps.cluster_columns]
    
    cluster_range = range( 1, 30 )
    cluster_errors = []

    for num_clusters in cluster_range:
        clusters = KMeans(n_clusters=num_clusters, init='k-means++')
        clusters.fit( Xpair )
        cluster_errors.append(clusters.inertia_)
        
    clusters_df = pd.DataFrame( { "num_clusters":cluster_range, "cluster_errors": cluster_errors } )   
    plt.figure(figsize=(12,6))
    plt.plot( clusters_df.num_clusters, clusters_df.cluster_errors, marker = "o" )   
    

    
def silhouette_analysis(hps,max_clusters):
    
    dataset = get_profiles(hps)
    values = dataset.values
    X_scaled = values[:,hps.cluster_columns]
    
    
    cluster_range = range(2, max_clusters)

    for n_clusters in cluster_range:
        # Create a subplot with 1 row and 2 columns
        fig, (ax1, ax2) = plt.subplots(1, 2)
        fig.set_size_inches(18, 7)

        # The 1st subplot is the silhouette plot
        # The silhouette coefficient can range from -1, 1 but in this example all
        # lie within [-0.1, 1]
        ax1.set_xlim([-1, 1])
        # The (n_clusters+1)*10 is for inserting blank space between silhouette
        # plots of individual clusters, to demarcate them clearly.
        ax1.set_ylim([0, len(X_scaled) + (n_clusters + 1) * 10])

        # Initialize the clusterer with n_clusters value and a random generator
        # seed of 10 for reproducibility.
        if(hps.use_spectral_clustering):
            clusterer = SpectralClustering(n_clusters=hps.clusters, affinity='nearest_neighbors',assign_labels='kmeans')
        else:    
            clusterer = KMeans (n_clusters=hps.clusters, init='k-means++')
        
        cluster_labels = clusterer.fit_predict( X_scaled )

        # The silhouette_score gives the average value for all the samples.
        # This gives a perspective into the density and separation of the formed
        # clusters
        silhouette_avg = silhouette_score(X_scaled, cluster_labels)
        print("For n_clusters =", n_clusters,
            "The average silhouette_score is :", silhouette_avg)

        # Compute the silhouette scores for each sample
        sample_silhouette_values = silhouette_samples(X_scaled, cluster_labels)

        y_lower = 10

        for i in range(n_clusters):
            # Aggregate the silhouette scores for samples belonging to
            # cluster i, and sort them
            ith_cluster_silhouette_values = \
              sample_silhouette_values[cluster_labels == i]

            ith_cluster_silhouette_values.sort()

            size_cluster_i = ith_cluster_silhouette_values.shape[0]
            y_upper = y_lower + size_cluster_i
            
            cmap = cm.get_cmap("Spectral")
            color = cmap(float(i) / n_clusters)

            ax1.fill_betweenx(np.arange(y_lower, y_upper),
                            0, ith_cluster_silhouette_values,
                            facecolor=color, edgecolor=color, alpha=0.7)

            # Label the silhouette plots with their cluster numbers at the middle
            ax1.text(-0.05, y_lower + 0.5 * size_cluster_i, str(i))

            # Compute the new y_lower for next plot
            y_lower = y_upper + 10  # 10 for the 0 samples

        ax1.set_title("The silhouette plot for the various clusters.")
        ax1.set_xlabel("The silhouette coefficient values")
        ax1.set_ylabel("Cluster label")

        # The vertical line for average silhoutte score of all the values
        ax1.axvline(x=silhouette_avg, color="red", linestyle="--")

        ax1.set_yticks([])  # Clear the yaxis labels / ticks
        ax1.set_xticks([-0.1, 0, 0.2, 0.4, 0.6, 0.8, 1])

        # 2nd Plot showing the actual clusters formed
        cmap = cm.get_cmap("Spectral")
        colors = cmap(cluster_labels.astype(float) / n_clusters)
        ax2.scatter(X_scaled[:, 0], X_scaled[:, 1], marker='.', s=30, lw=0, alpha=0.7,
                  c=colors)

        # Labeling the clusters
#        centers = clusterer.cluster_centers_
        # Draw white circles at cluster centers
#         ax2.scatter(centers[:, 0], centers[:, 1],
#                   marker='o', c="white", alpha=1, s=200)

#         for i, c in enumerate(centers):
#             ax2.scatter(c[0], c[1], marker='$%d$' % i, alpha=1, s=50)

        ax2.set_title("The visualization of the clustered data.")
        ax2.set_xlabel("Feature space for the 1st feature")
        ax2.set_ylabel("Feature space for the 2nd feature")

        plt.suptitle(("Silhouette analysis for clustering on sample data "
                    "with n_clusters = %d" % n_clusters),
                   fontsize=14, fontweight='bold')

        plt.show()    
        

# def get_data(file_name,hps):
#     columns = ['user','current','start','session_id',
#                'prev_session_length','avg_session_length',
#                'gender','age','country','registered',
#                'track_duration','times_played','artist','track','session_length']
#     complete_files = glob.glob(file_name)
#     dataset = pd.concat((pd.read_csv(f,names=columns,sep='\t') for f in complete_files))
    
#     if(hps.filter_outliers):
#         df_perc = np.percentile(dataset.session_length, [hps.upper_limit])
#         dataset =  dataset[dataset.session_length < df_perc[0]]
#         dataset =  dataset[dataset.prev_session_length < df_perc[0]]
        
# #         df_perc = np.percentile(dataset.session_length, [hps.lower_limit])
# #         dataset =  dataset[dataset.session_length > df_perc[0]]
# #         dataset =  dataset[dataset.prev_session_length > df_perc[0]]
        
#     dataset = dataset.dropna()
#     #dataset = dataset.sort_values(by=['start'])  
    
#     values = dataset.values
#     X = values[:,:-1]
#     y = values[:,-1]    
    
#     #3D - samples,timesteps,features
#     X = X.reshape((X.shape[0], 1, X.shape[1]))
#     return X,y



Overwriting util.py
